<a href="https://colab.research.google.com/github/Lucia1009/TFG-Plataforma-para-la-prediccion-de-potencia-electrica-generada-en-parques-eolicos/blob/desarrollo/modelos/comparacion_predicciones_diarias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash -q

In [67]:
from google.colab import drive

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle

from datetime import date, datetime
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, max_error
from sklearn.impute import KNNImputer


In [3]:
drive.mount('/content/drive', force_remount=True)

reales=pd.read_json('/content/drive/MyDrive/TFG/datos santiago/RedMeteorológica_Santiago-EOAS_diezminutal_28-03-2025_to_10-04-2025.json')

# #cargo las prediccines con pickle
# preds_28=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-03-28.pkl', 'rb'))
# preds_29=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-03-29.pkl', 'rb'))
# preds_30=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-03-30.pkl', 'rb'))
# preds_31=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-03-31.pkl', 'rb'))

# preds_04=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-04-04.pkl', 'rb'))
# preds_05=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-04-05.pkl', 'rb'))
# preds_06=pickle.load(open('/content/drive/MyDrive/TFG/datos santiago/forecast_2025-04-06.pkl', 'rb'))

# predicciones=[preds_28,preds_29,preds_30,preds_31,preds_04,preds_05,preds_06]


# abrir todos los archivos de datos de la carpeta /content/drive/MyDrive/TFG/datos santiago que empiecen por forecast y guardarlos en predicciones
import os
predicciones = []
for filename in os.listdir('/content/drive/MyDrive/TFG/datos santiago'):
    if filename.startswith('forecast'):
        with open(os.path.join('/content/drive/MyDrive/TFG/datos santiago', filename), 'rb') as f:
            predicciones.append(pickle.load(f))



# Cambio el nombre de las columnas para todos los dicionarios
for preds in predicciones:
  for modelo in preds:
    preds[modelo] = pd.DataFrame(preds[modelo])
    preds[modelo].rename(columns={'date': 'Fecha', 'temperature_2m': 'temperatura', 'wind_speed_10m': 'wind_speed', 'wind_direction_10m':'wind_direction'}, inplace=True)

display(predicciones[0]['best_match'].head())


Mounted at /content/drive


,Fecha,temperatura,wind_speed,wind_direction,wd_sin_10m,wd_cos_10m
0,2025-03-28 00:00:00+00:00,7.5825,7.412853,29.054508,0.485642,0.874158
1,2025-03-28 01:00:00+00:00,6.5825,8.217153,28.810698,0.481917,0.876217
2,2025-03-28 02:00:00+00:00,6.0825,7.903619,30.068504,0.501035,0.865427
3,2025-03-28 03:00:00+00:00,5.5325,7.787991,33.690102,0.554701,0.832050
4,2025-03-28 04:00:00+00:00,5.2825,7.993297,35.837746,0.585492,0.810678


In [4]:
"""Trato los datos reales de santiago"""

unique_codigos = reales['Código'].unique().tolist()
datos = pd.DataFrame(columns=['Fecha']+unique_codigos )

for codigo in unique_codigos:

  variable= reales[reales['Código'] ==codigo][['Fecha','Código','Valor', 'Código de validación']]
  variable=variable[variable['Código de validación']==1]

  datos['Fecha'] = reales['Fecha'].unique()

  merged_data = datos.merge(variable[['Fecha', 'Valor']], on='Fecha', how='left')
  datos[codigo] = merged_data['Valor']

datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%d/%m/%Y %H:%M')
datos['Fecha']=datos['Fecha'].dt.tz_localize(None)
datos.set_index('Fecha', inplace=True)


# display(datos.head(10))
print(datos.shape)

# Datos con la media de cada hora
reales=datos.resample('h').mean()

#renombramos columnas
reales = reales.rename(columns={'TA_AVG_1.5m': 'temperatura', 'VV_AVG_10m': 'wind_speed', 'DV_AVG_10m': 'wind_direction'})

display(reales.head(10))
print(reales.shape)

(1907, 3)


,wind_direction,temperatura,wind_speed
Fecha,,,
2025-03-28 00:00:00,30.200000,8.040000,16.200000
2025-03-28 01:00:00,36.333333,7.650000,15.683333
2025-03-28 02:00:00,36.500000,7.535000,16.766667
2025-03-28 03:00:00,31.000000,6.873333,17.233333
2025-03-28 04:00:00,25.000000,6.365000,18.716667
2025-03-28 05:00:00,41.166667,5.895000,13.650000
2025-03-28 06:00:00,36.166667,5.776667,14.300000
2025-03-28 07:00:00,36.333333,5.805000,11.950000
2025-03-28 08:00:00,45.333333,7.461667,11.766667


(319, 3)


In [5]:
"""Hago un gráfico con las predicciones y los datos reales"""

atribs=['temperatura', 'wind_speed', 'wind_direction']
lista_modelos=list(predicciones[0].keys())
#Lista de colores para los distintos días
colores=['blue', 'green', 'yellow', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

app = Dash('app1')

app.layout = html.Div([

  dcc.Dropdown(lista_modelos, lista_modelos[0], id='modelo', clearable=False),
  dcc.Dropdown(atribs, atribs[0], id='atributo', clearable=False),

  dcc.Graph(id='graph')
])


@app.callback(
    Output('graph', 'figure'),
    Input('modelo', 'value'),
    Input('atributo', 'value'))
def update_output_div(modelo, atributo):

  fig = go.Figure()


  fig.add_trace(go.Scatter(y=reales[atributo], x=reales.index, mode='lines', marker=dict(color='red'), name='Historicas estacion'))


  for preds, color in zip(predicciones, colores):

    reales_cp = reales.copy()
    reales_cp=reales_cp.reset_index()

    df_pred = preds[modelo].copy()
    df_pred['Fecha'] = pd.to_datetime(df_pred['Fecha']).dt.tz_localize(None)


    datos_trabajo_model = reales_cp.merge(
        df_pred,
        on="Fecha",
        how="inner",
        suffixes=("", f"_{modelo}")
    )


    fig.add_trace(go.Scatter(
        y=datos_trabajo_model[f"{atributo}_{modelo}"],
        x=datos_trabajo_model["Fecha"],
        mode='lines',
        marker=dict(color=color),
        opacity=0.5,
        name=datos_trabajo_model["Fecha"][0].strftime('%d-%m-%Y')
    ))



  return fig

if __name__ == '__main__':
    app.run(debug=True, port=8081)

<IPython.core.display.Javascript object>

# Error por día para cada conjunto de datos

In [9]:
for dia in predicciones:
  # Tomo la fecha de la primera fila de pred
  fecha=dia["best_match"]["Fecha"][0].strftime('%d-%m-%Y')

  for pred in predicciones:
    if fecha in pred["best_match"]["Fecha"].dt.strftime('%d-%m-%Y').tolist():
      for modelo in lista_modelos:
        print(fecha, modelo)

        for atributo in atribs:

          reales_fecha=reales[reales.index.strftime('%d-%m-%Y')==fecha][atributo]
          pred[modelo]['Fecha'] = pd.to_datetime(pred[modelo]['Fecha'])
          pred_fecha=pred[modelo][pred[modelo]['Fecha'].dt.strftime('%d-%m-%Y')==fecha][atributo]

          if not reales_fecha.empty and not pred_fecha.empty:

            imputer = KNNImputer(n_neighbors=3)
            reales_fecha = imputer.fit_transform(reales_fecha.values.reshape(-1, 1)).flatten()
            pred_fecha = imputer.fit_transform(pred_fecha.values.reshape(-1, 1)).flatten()

            #me quedo sólo con los datos reales de las horas que tenga pred_fecha
            reales_fecha = reales_fecha[:len(pred_fecha)]

            rmse=math.sqrt(mean_squared_error(reales_fecha, pred_fecha))
            mae=mean_absolute_error(reales_fecha, pred_fecha)
            r2=r2_score(reales_fecha, pred_fecha)
            max_error_value=max_error(reales_fecha, pred_fecha)

            print(atributo)
            print('rmse', rmse, 'mae', mae, 'r2', r2, 'max_error', max_error_value)
          else:
            print(f"No hay datos para {fecha}, {modelo}, {atributo}. No se puede hacer el cálculo")
        print()
      print()

28-03-2025 best_match
temperatura
rmse 0.9298206725967421 mae 0.8029861405160692 r2 0.9329245406411093 max_error 1.5808330663045247
wind_speed
rmse 5.989053502094402 mae 5.125063245826297 r2 -3.83354502145297 max_error 10.723370043436685
wind_direction
rmse 32.91879186888968 mae 17.191926831669278 r2 -0.6339284054890486 max_error 123.6901044845581

28-03-2025 ecmwf_ifs04
No hay datos para 28-03-2025, ecmwf_ifs04, temperatura. No se puede hacer el cálculo
No hay datos para 28-03-2025, ecmwf_ifs04, wind_speed. No se puede hacer el cálculo
No hay datos para 28-03-2025, ecmwf_ifs04, wind_direction. No se puede hacer el cálculo

28-03-2025 gfs_global
temperatura
rmse 1.879633296367359 mae 1.5501528325345781 r2 0.7258984321472048 max_error 3.1276669947306317
wind_speed
rmse 8.666098943645423 mae 8.135939039124382 r2 -9.120376487402112 max_error 13.22150561014811
wind_direction
rmse 35.1233217679643 mae 22.50715045266681 r2 -0.8601005084153202 max_error 127.14676713943481

28-03-2025 jma_gsm


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



wind_speed
rmse 9.257079240127535 mae 7.634508747193549 r2 -0.9294216282528038 max_error 19.33999991416931
wind_direction
rmse 67.54688800160373 mae 42.41631592644586 r2 -5.7301231318317285 max_error 171.00010299682617

05-04-2025 ecmwf_ifs04
No hay datos para 05-04-2025, ecmwf_ifs04, temperatura. No se puede hacer el cálculo
No hay datos para 05-04-2025, ecmwf_ifs04, wind_speed. No se puede hacer el cálculo
No hay datos para 05-04-2025, ecmwf_ifs04, wind_direction. No se puede hacer el cálculo

05-04-2025 gfs_global
temperatura
rmse 1.07196307028586 mae 0.9362778260972767 r2 0.44108070302544766 max_error 2.0756663386027014
wind_speed
rmse 6.643374916626939 mae 5.632247748639848 r2 0.006296762514143683 max_error 10.941550254821777
wind_direction
rmse 22.549502404626633 mae 14.256398306952581 r2 0.24995685642963228 max_error 78.12891642252605

05-04-2025 jma_gsm
temperatura
rmse 1.7568409274713797 mae 1.4658333810170492 r2 -0.5012545501866388 max_error 3.2150003814697268
wind_speed
rmse

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



wind_speed
rmse 5.130913337891458 mae 4.400960972574022 r2 0.40725412716045384 max_error 12.752571868896485
wind_direction
rmse 21.601914463078934 mae 14.606249279446073 r2 0.311669832475701 max_error 71.97119903564453

05-04-2025 ecmwf_ifs04
No hay datos para 05-04-2025, ecmwf_ifs04, temperatura. No se puede hacer el cálculo
No hay datos para 05-04-2025, ecmwf_ifs04, wind_speed. No se puede hacer el cálculo
No hay datos para 05-04-2025, ecmwf_ifs04, wind_direction. No se puede hacer el cálculo

05-04-2025 gfs_global
temperatura
rmse 1.3569107014345303 mae 1.145027969148424 r2 0.10444555989077564 max_error 3.075667292277018
wind_speed
rmse 8.33872959300818 mae 7.006136026647357 r2 -0.5655933272143212 max_error 14.700103569030762
wind_direction
rmse 26.978782874882572 mae 17.011652946472168 r2 -0.07363590168874201 max_error 91.4946517944336

05-04-2025 jma_gsm
temperatura
rmse 0.9432564370276361 mae 0.814166595141093 r2 0.5672381073348103 max_error 1.6683331425984704
wind_speed
rmse 8.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



temperatura
rmse 1.5341902025095955 mae 1.2759862195120917 r2 0.8654171114200417 max_error 3.182666422526042
wind_speed
rmse 6.977143791128296 mae 6.053709461953905 r2 -3.2865658788060887 max_error 11.907003529866536
wind_direction
rmse 130.1499827252012 mae 111.65124871995715 r2 -1.480083198918687 max_error 255.33333333333331

06-04-2025 jma_gsm
temperatura
rmse 1.914543202997676 mae 1.5899304681354096 r2 0.7904142373116656 max_error 3.418334096272787
wind_speed
rmse 6.193843123294902 mae 5.110129613098171 r2 -2.3781160007774367 max_error 12.518668238321942
wind_direction
rmse 97.52427467060558 mae 86.17009862263997 r2 -0.39252653199687404 max_error 170.16666666666669

06-04-2025 icon_global
temperatura
rmse 2.9332593046941238 mae 2.5079581948121388 r2 0.5080371039144633 max_error 5.340166365305583
wind_speed
rmse 5.397727164859048 mae 4.465957423051198 r2 -1.5655235964635112 max_error 10.53154862721761
wind_direction
rmse 111.48598829440455 mae 76.23161686791313 r2 -0.819778781562584

In [80]:
metrics = ['rmse', 'mae', 'r2', 'max_error']



# Método que obtiene el error de la dirección a través del error del seno y el coseno
def diferencia_angulos(sin_real, cos_real, sin_pagina, cos_pagina):

  producto_escalar = sin_real * sin_pagina + cos_real * cos_pagina
  producto_escalar = max(-1.0, min(1.0, producto_escalar))
  angulo_rad = math.acos(producto_escalar)
  angulo_deg = math.degrees(angulo_rad)
  return angulo_deg




# Función que obtiene el modelo cuyas métricas son mejores para todos los atributos
def max_metricas(scores):

    # Recolectar todos los valores por métrica y atributo
    metricas_valores ={atributo: {metrica: [] for metrica in metrics} for atributo in atribs}

    for modelo in lista_modelos:
        for atributo, metrica_dict in scores[modelo].items():
            for metrica, valor in metrica_dict.items():
                metricas_valores[atributo][metrica].append(valor)

    # Normalizar y calcular el puntaje por modelo
    puntajes = {}
    for modelo in lista_modelos:
        puntaje_total = 0
        cuenta_metricas = 0
        for atributo, metrica_dict in scores[modelo].items():
            for metrica, valor in metrica_dict.items():
                valores = metricas_valores[atributo][metrica]
                minimo = min(valores)
                maximo = max(valores)
                if maximo != minimo:
                    valor_normalizado = (valor - minimo) / (maximo - minimo)
                else:
                    valor_normalizado = 1.0  # todos iguales => máxima puntuación
                puntaje_total += valor_normalizado
                cuenta_metricas += 1
        # Convertimos el puntaje a escala de 0 a 100
        puntajes[modelo] = (puntaje_total / cuenta_metricas) * 100

    best_modelo = max(puntajes, key=puntajes.get)

    return scores[best_modelo], best_modelo




# Método que obtiene los datasets de predicciones que contienen predicciones para una determinada fecha
# y devuelve las scores del modelo con mejores resultados para cada uno de los datasets
def obtener_scores_por_fechas(fecha, reales, preds):
    scores = {}
    mejores_modelos = []

    fecha_str = fecha.strftime('%d-%m-%Y')

    for i, dia in enumerate(preds):
        if "best_match" not in dia or dia["best_match"].empty:
            continue

        contiene_fecha = False
        for modelo in lista_modelos:
            if modelo in dia:
                fechas_modelo = dia[modelo]['Fecha'].dt.strftime('%d-%m-%Y')
                if fecha_str in fechas_modelo.values:
                    contiene_fecha = True
                    break

        if not contiene_fecha:
            continue  # Este dataset de predicción no incluye esa fecha

        scores_temp = {}

        for modelo in lista_modelos:
            scores_temp[modelo] = {}

            for atributo in atribs:


                reales_fecha = reales[reales.index.strftime('%d-%m-%Y') == fecha_str][atributo]
                dia[modelo]['Fecha'] = pd.to_datetime(dia[modelo]['Fecha'])
                pred_fecha = dia[modelo][dia[modelo]['Fecha'].dt.strftime('%d-%m-%Y') == fecha_str][atributo]

                if not reales_fecha.empty and not pred_fecha.empty:

                    imputer = KNNImputer(n_neighbors=3)
                    reales_fecha = imputer.fit_transform(reales_fecha.values.reshape(-1, 1)).flatten()
                    pred_fecha = imputer.fit_transform(pred_fecha.values.reshape(-1, 1)).flatten()

                    reales_fecha = reales_fecha[:len(pred_fecha)]

                    if atributo == 'wind_direction':
                      error=np.mean([diferencia_angulos(np.sin(reales_fecha[k]), np.cos(reales_fecha[k]), np.sin(pred_fecha[k]), np.cos(pred_fecha[k])) for k in range(len(reales_fecha))])
                      scores_temp[modelo][atributo] = {
                          'rmse': error,
                          'mae': float('nan'),
                          'r2': float('nan'),
                          'max_error': float('nan')
                      }
                      continue

                    rmse = math.sqrt(mean_squared_error(reales_fecha, pred_fecha))
                    mae = mean_absolute_error(reales_fecha, pred_fecha)
                    r2 = r2_score(reales_fecha, pred_fecha)
                    max_error_value = max_error(reales_fecha, pred_fecha)

                    scores_temp[modelo][atributo] = {
                        'rmse': rmse,
                        'mae': mae,
                        'r2': r2,
                        'max_error': max_error_value
                    }
                else:
                    scores_temp[modelo][atributo] = {
                        'rmse': float('nan'),
                        'mae': float('nan'),
                        'r2': float('nan'),
                        'max_error': float('nan')
                    }

        # Guardamos los scores del mejor modelo para este dataset
        fecha_clave=dia["best_match"]["Fecha"][0].strftime('%d-%m-%Y')
        # fecha_clave = f'pred_{i}'
        scores[fecha_clave], best = max_metricas(scores_temp)
        mejores_modelos.append(best)

    return scores, mejores_modelos

In [83]:
"""Gráfica que toma las scores del modelo con las mejores métricas que contienen predicciones para el día seleccionado por el usuario. Se representan las scores para todos los modelos que predicen ese día"""

seleccionados = ['hist_pag', 'datos_hora']
max_date=reales.index.max()


app = Dash('app1')

app.layout = html.Div([
    dcc.DatePickerSingle(
        id='fecha_comparacion',
        min_date_allowed=date(2025, 3, 28),
        max_date_allowed=max_date,
        first_day_of_week=1,
        date=date(2025, 3, 28),
        display_format='D-M-Y'
    ),
    dcc.Graph(id='graph')
])


@app.callback(
    Output('graph', 'figure'),
    Input('fecha_comparacion', 'date'))
def update_output_div(fecha):
    if fecha is None:
        return go.Figure()

    fecha = datetime.fromisoformat(fecha).date()

    scores, mejores_modelos = obtener_scores_por_fechas(fecha, reales, predicciones)

    fig = make_subplots(
        rows=len(metrics), cols=1,
        subplot_titles=[f'{metric.upper()} Scores per Variable and prediction set' for metric in metrics],
        shared_xaxes=True
    )

    model_colors = px.colors.sample_colorscale(
        "Viridis",
        [i / (len(scores) - 1) if len(scores) > 1 else 0.5 for i in range(len(scores))]
    )

    dias = list(scores.keys())

    for idx, dia in enumerate(dias):
        modelo = mejores_modelos[idx]

        for metric_index, metric in enumerate(metrics):
            y_values = []

            for variable in atribs:
                valor = scores[dia][variable].get(metric, None)
                if valor is None or np.isnan(valor):
                    valor = 0
                y_values.append(valor)

            fig.add_trace(
                go.Bar(
                    x=atribs,
                    y=y_values,
                    name=f'{dia} ({modelo})',
                    marker_color=model_colors[idx],
                    text=[f'{val:.2f}' for val in y_values],
                    textposition='auto'
                ),
                row=metric_index + 1,
                col=1
            )

            fig.update_yaxes(title_text=metric.upper(), row=metric_index + 1, col=1)

    fig.update_xaxes(title_text="Variables", row=len(metrics), col=1)

    fig.update_layout(
        barmode='group',
        height=1000,
        width=1200,
        legend_title_text='Modelos'
    )

    return fig



if __name__ == '__main__':
    app.run(debug=True, port=8082)


<IPython.core.display.Javascript object>